# Google Calendar Assistant with with Llama 3.2 3B Tool Calling

This notebook showcases how to go about building a digital assistant to schedule meetings with the Llama 3.2 3B model. The core concepts used to implement this are Prompt Engineering and Tool Calling. This demo shows how Llama can be used to interact with 3rd party apps like Google Contacts & Google Calendar and schedule a meeting requested by the user. Even though we are using prompt engineering to achieve this, the approach described doesn't degrade the model's ability to answer general queries. This approach can extended to perform other tasks in a similar manner without affecting the quality of other tasks


## Approach

Instead of using a complex system prompt with multiple conditions & expecting Llama to perform various tasks accurately out of the box, the approach here is to treat this as a 2 step process
- Determine user intent - Task classification
- Take action for the specific task using Tool Calling



In the diagram shown below,
- system prompt 1 determines the classfication of the query
- In steps 2 & 3, we classify the task being requested.
- system prompt 2 is chosen based on the classification result
- Steps 4 & 5 implement the classified task.
- For the sake of demo, we show 2 classes: General & Meeting

![Tool Calling Flow Diagram](./assets/flow_diagram.png)

Both these tasks have a specific prompt. We use the same model with different system prompts depending on the classification result.
Additionally, this demo also showcases how Llama can be used to do double tool calling with 1 prompt. In the case of Meeting, Llama returns 2 function calls in Step 5
```
<function=google_contact>{{"name": "John Constantine"}}</function>
<function=google_calendar>{{"date": "Mar 31 ", "time": "5:30 pm", "attendees": "John Constantine"}}</function>
```

Actions based on tool calling output
- The google_contact function call returned by the model is used to call [People API](https://developers.google.com/people) to look up the email address of the person of interest
- The email address from the previous step is used to call [Calendar API](https://developers.google.com/calendar) along with the other information in the google_calendar toolcall output returned by the model 

The end result is that a google meeting is scheduled with the person of interest at the date & time specified


## Load Llama 3.2-3B-Instruct model

This demo also intends to show that tool calling can be done effectively even with the 3B model

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Llama-3.2-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(model_id)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Define functions to access People (`google_contact`) and Google Calendar (`google_calendar`) API

**Note!!!!**
Accessing Google APIs require you to first get credentials (This is a one time process) 

Store your credentials in `credentials.json` Please refer to the [instructions](https://developers.google.com/workspace/guides/create-credentials) here to get the credentials.

I followed the the steps in [service account](https://developers.google.com/identity/protocols/oauth2/service-account#creatinganaccount) to get my credentials.

Accessing Google APIs also require you to get **authentication token** in addition to the credentials. The functions defined below `google_contact` and `google_calendar` also include the logic to generate & refresh the authentication token.

To get the authentication token for using the People & Google Calendar API, we need a runnable browser available on the machine where you execute these functions for the **FIRST TIME** only. You will need to authenticate using your browser for the first time. Executing these functions will generate a `token_contacts.json` & `token_calendar.json`

For subsequent calls, you don't need a runnable browser

#### Install required libraries

In [ ]:
! pip install google-api-python-client google-auth-oauthlib

In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime, timedelta


SCOPES = ["https://www.googleapis.com/auth/contacts.readonly"]

TOKEN = "token_contacts.json"


def google_contact(name):
  """Returns the email address in Google contacts for the given name.
  """
  creds = None

  # The file token_contacts.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists(TOKEN):
    creds = Credentials.from_authorized_user_file(TOKEN, SCOPES)

  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open(TOKEN, "w") as token:
      token.write(creds.to_json())

  try:
    service = build("people", "v1", credentials=creds)

    # Call the People API
    results = (
        service.people()
        .connections()
        .list(
            resourceName="people/me",
            pageSize=10,
            personFields="names,emailAddresses",
        )
        .execute()
    )
    connections = results.get("connections", [])

    # Build a dictionary of name & email address
    db = {}
    for person in connections:
      names = person.get("names", [])
      if names:
        n = names[0].get("displayName")
        
      email = person.get("emailAddresses", [])
      db[n] = email[0].get('value')
    
    return db[name]
  except HttpError as err:
    print(err)


CALSCOPES = ["https://www.googleapis.com/auth/calendar"]

CALTOKEN = "token_calendar.json"

def google_calendar(date, time, attendees):
  """Creates a meeting invite using Google Calendar API.
  """
  creds = None
  # The file token_calendar.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists(CALTOKEN):
    creds = Credentials.from_authorized_user_file(CALTOKEN, CALSCOPES)
        
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", CALSCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open(CALTOKEN, "w") as token:
      token.write(creds.to_json())

  name, email = attendees["name"], attendees["email"]
  time_str = date + ', 2025 ' + time
  # Convert the time string to a datetime object
  dt_obj = datetime.strptime(time_str, '%b %d, %Y %I:%M %p')
  dt_obj_end =  dt_obj + timedelta(minutes=30)
  event = {
    'summary': 'Meeting: Ankith | ' + name,
    'location': 'MPK 21',
    'description': 'Sync up meeting',
    'start': {
      'dateTime': dt_obj.strftime('%Y-%m-%dT%H:%M:%S'),
      'timeZone': 'America/Los_Angeles',
    },
    'end': {
      f'dateTime': dt_obj_end.strftime('%Y-%m-%dT%H:%M:%S'),
      'timeZone': 'America/Los_Angeles',
    },
    'attendees': [
      {'email': email},
    ],
    'reminders': {
      'useDefault': False,
      'overrides': [
        {'method': 'email', 'minutes': 24 * 60},
        {'method': 'popup', 'minutes': 10},
      ],
    },
  }
  
  service = build("calendar", "v3", credentials=creds)

  # Create a meeting invite
  event = service.events().insert(calendarId='primary', body=event).execute()
  
  result = f"\nEvent created with {name} with email {email} on {date} at {time}: \n {event.get('htmlLink')}"
  return result


### Function to process tool calling output and call Google APIs

In [2]:
import json

functions = {
  "google_contact": google_contact,
  "google_calendar": google_calendar
}


def process_llama_output(llama_output):
  """
  Function to process the tool calling output & make the actual Google API call
  """

  result = ""
  prefix = "<function="
  suffix = "</function>"
  start, N = 0, len(llama_output)
  count = 0
  email = None

  # In this example, we are expecting Llama to produce 2 tool calling outputs
  while count < 2:
    begin, end = llama_output[start:].find(prefix) + len(prefix) + start, llama_output[start:].find(suffix) + start
    func_name, params = llama_output[begin:end-1].split(">{")
    end += len(suffix)
    start = end
    count += 1
    params = json.loads(params.strip())
    
    if not email and func_name in functions:
      email = functions[func_name](**params)
    
    elif email and func_name in functions:
      attendees = {}
      attendees["name"] = params["attendees"]
      attendees["email"] = email
      params["attendees"] = attendees

      result += "\n-----------------------------------------------------\n"
      result += functions[func_name](**params)
      return result


### Functions for calling Llama for the 2 tasks: general & setup meeting

In [3]:
def general_query(user_prompt, result):
  """
  Function to call Llama for general queries
  """
  
  result += "\n-----------------------------------------------------\n"
  result += "SECOND PASS"
  result += "\n-----------------------------------------------------\n"

  max_new_tokens = 100
  SYSTEM_PROMPT1 = f"""

  You are a helpful assistant. Answer the query in {max_new_tokens} words
  """

  prompt = SYSTEM_PROMPT1 + f"\n\nQuestion: {user_prompt} \n" + "\n Answer:\n"
  input_prompt_length = len(prompt)

  input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

  output = model.generate(**input_ids, max_new_tokens=max_new_tokens)

  llama_output = tokenizer.decode(output[0], skip_special_tokens=True)


  result += llama_output

  return result


def setup_meeting(user_prompt, result):
  """
  Function to call Llama for setting up meeting using Google APIs
  """
  
  result += "\n-----------------------------------------------------\n"
  result += "SECOND PASS"
  result += "\n-----------------------------------------------------\n"

  max_new_tokens = 100
  SYSTEM_PROMPT2 = """

  You are a helpful assistant.
  Here are the tools you have access to: google_contact and google_calendar which you can use for ONLY setting up meetings:
  Use the function 'google_contact' to: look up a name and get the email address
  {
    "name": "google_contact",
    "description": "Get email address",
    "parameters": {
      "name": {
        "param_type": "string",
        "description": "name",
        "required": true
      }
    }
  }
  Use the function 'google_calendar' to: schedule a meeting
  {
    "name": "google_calendar",
    "description": "Schedule a meeting",
    "parameters": {
      "date": {
        "param_type": "string",
        "description": "date",
        "required": true
      },
      "time"": {
        "param_type": "string",
        "description": "time",
        "required": true
      },
      "attendees"": {
        "param_type": "string",
        "description": "name",
        "required": true
      }
    }
  }
  Identify the name in the query, lookup the name using 'google_contact' to find the email address of the contact 
  and then use 'google_calndar' to schedule a meeeting with this person.
  DO NOT reply with any other reasoning or exaplanation
  ONLY reply in the following format with no prefix or suffix:
  <function=function_name_being_used>{{"example_name": "example_value"}}</function>
  Reminder:
  - ONLY return the function call and don't reply anything else
  - DO NOT give any explanation
  - I REPEAT ONLY return function calls
  - Function calls MUST follow the specified format, start with <function= and end with </function>
  - Required parameters MUST be specified
  - First return google_contact and then google_calendar
  """


  prompt = SYSTEM_PROMPT2 + f"\n\nQuestion: {user_prompt} \n"

  input_prompt_length = len(prompt)

  input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

  output = model.generate(**input_ids, max_new_tokens=max_new_tokens)

  llama_output = tokenizer.decode(output[0], skip_special_tokens=True)


  result += llama_output


  result += "\n\n\n"
  result += process_llama_output(llama_output[input_prompt_length:])
  return result

## Demo with a general query

To help explain the final output, we also print the system prompts being used in both the calls to Llama

In [4]:
user_prompt = "Tell me about Paris"

In [5]:
SYSTEM_PROMPT = """

  Classify the given prompt
  A message can be classified as one of the following categories: meeting, general.

  Examples:
  - meeting: "Can you schedule a meeting with Ankith on Dec 1 at 1 pm."
  - general: "Tell me about San Francisco"

  Reminder:
  - ONLY return the classification label and NOTHING else
  - DO NOT give any explanation or examples
  """

prompt = SYSTEM_PROMPT + f"\n\nQuestion: {user_prompt} \n" + "\n Answer:\n"

input_prompt_length = len(prompt)

In [8]:

result = "\n-----------------------------------------------------\n"
result += "FIRST PASS"
result += "\n-----------------------------------------------------\n"

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(**input_ids, max_new_tokens=2)

llama_output = tokenizer.decode(output[0], skip_special_tokens=True)

result += llama_output
result += "\n"


classification = llama_output[input_prompt_length:].strip()


if  "meeting" in classification:
    result = setup_meeting(prompt, result)
elif "general" in classification:
    result = general_query(user_prompt, result)
print(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



-----------------------------------------------------
FIRST PASS
-----------------------------------------------------


  Classify the given prompt
  A message can be classified as one of the following categories: meeting, general.

  Examples:
  - meeting: "Can you schedule a meeting with Ankith on Dec 1 at 1 pm."
  - general: "Tell me about San Francisco"

  Reminder:
  - ONLY return the classification label and NOTHING else
  - DO NOT give any explanation or examples
  

Question: Tell me about Paris 

 Answer:
 general 



-----------------------------------------------------
SECOND PASS
-----------------------------------------------------


  You are a helpful assistant. Answer the query in 100 words
  

Question: Tell me about Paris 

 Answer:
Paris, the City of Light, is the capital of France, known for its stunning architecture, art, fashion, and romance. The city is home to iconic landmarks like the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum, which houses the

In [9]:
user_prompt = "Can you describe how an internal combustion engine works "

prompt = SYSTEM_PROMPT + f"\n\nQuestion: {user_prompt} \n" + "\n Answer:\n"

input_prompt_length = len(prompt)

In [10]:

result = "\n-----------------------------------------------------\n"
result += "FIRST PASS"
result += "\n-----------------------------------------------------\n"

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(**input_ids, max_new_tokens=2)

llama_output = tokenizer.decode(output[0], skip_special_tokens=True)

result += llama_output
result += "\n"


classification = llama_output[input_prompt_length:].strip()


if  "meeting" in classification:
    result = setup_meeting(prompt, result)
elif "general" in classification:
    result = general_query(user_prompt, result)
print(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



-----------------------------------------------------
FIRST PASS
-----------------------------------------------------


  Classify the given prompt
  A message can be classified as one of the following categories: meeting, general.

  Examples:
  - meeting: "Can you schedule a meeting with Ankith on Dec 1 at 1 pm."
  - general: "Tell me about San Francisco"

  Reminder:
  - ONLY return the classification label and NOTHING else
  - DO NOT give any explanation or examples
  

Question: Can you describe how an internal combustion engine works  

 Answer:
general



-----------------------------------------------------
SECOND PASS
-----------------------------------------------------


  You are a helpful assistant. Answer the query in 100 words
  

Question: Can you describe how an internal combustion engine works  

 Answer:
An internal combustion engine is a type of heat engine that generates power by burning fuel, typically gasoline or diesel, inside a combustion chamber within the e

## Demo with a set up meeting query, which uses tool calling & calls Google APIs

To help explain the final output, we also print the system prompts being used in both the calls to Llama

In [53]:
user_prompt = "Schedule a meeting with John Constantine on Mar 31 at 5:30 pm"

In [54]:
SYSTEM_PROMPT = """

  Classify the given prompt
  A message can be classified as one of the following categories: meeting, general.

  Examples:
  - meeting: "Can you schedule a meeting with Ankith on Dec 1 at 1 pm."
  - general: "Tell me about San Francisco"

  Reminder:
  - ONLY return the classification label and NOTHING else
  - DO NOT give any explanation or examples
  """

prompt = SYSTEM_PROMPT + f"\n\nQuestion: {user_prompt} \n" + "\n Answer:\n"

input_prompt_length = len(prompt)

In [55]:

result = "\n-----------------------------------------------------\n"
result += "FIRST PASS"
result += "\n-----------------------------------------------------\n"

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(**input_ids, max_new_tokens=2)

llama_output = tokenizer.decode(output[0], skip_special_tokens=True)

result += llama_output
result += "\n"


classification = llama_output[input_prompt_length:].strip()

if  "meeting" in classification:
    result = setup_meeting(prompt, result)
elif "general" in classification:
    result = general_query(user_prompt, result)
print(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



-----------------------------------------------------
FIRST PASS
-----------------------------------------------------


  Classify the given prompt
  A message can be classified as one of the following categories: meeting, general.

  Examples:
  - meeting: "Can you schedule a meeting with Ankith on Dec 1 at 1 pm."
  - general: "Tell me about San Francisco"

  Reminder:
  - ONLY return the classification label and NOTHING else
  - DO NOT give any explanation or examples
  

Question: Schedule a meeting with John Constantine on Mar 31 at 5:30 pm 

 Answer:
  meeting

-----------------------------------------------------
SECOND PASS
-----------------------------------------------------


  You are a helpful assistant.
  Here are the tools you have access to: google_contact and google_calendar which you can use for ONLY setting up meetings:
  Use the function 'google_contact' to: look up a name and get the email address
  {
    "name": "google_contact",
    "description": "Get email add

## Result

In your Google Calendar, you should see an invite generated as shown below

<img src="./assets/google_calendar.png" alt="Google Calendar Invite" width="300" height="200">